## **Practico 2: Entrenamos modelos**

1) A partir del grafo de elliptic dataset provisto en la librería torch-geometric, generar una versión de tabla csv, a partir de la cual se podría haber construido el mismo. Comprender en profundidad como se construye un grafo a partir de los datos de las transacciones y viceversa.

2) (Tarea mas importante) Tomar como punto de partida el elliptic dataset provisto en la librería torch-geometric y comenzar una prueba de concepto buscando entrenar un modelo de machine learning capaz de predecir el fraude. Pueden trabajar con el archivo en formato grafo o csv.

## Importar librerias

In [29]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import networkx as nx
import random
import torch_geometric

sns.set_style("whitegrid")
sns.set_context("talk")

from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

## 1- Generar CSV a partir de un grafo

Se comienza por importar el grafo de la librería torch-geometric:

In [2]:
from torch_geometric.datasets import EllipticBitcoinDataset

dataset = EllipticBitcoinDataset(root='data/elliptic')
data = dataset[0]  # Es un solo grafo

Se hecha un vistazo a los principales atributos de dicho grafo:

In [3]:
# Mostrar todos los atributos disponibles
print("Atributos disponibles en el grafo:")
print(data.keys())

Atributos disponibles en el grafo:
['edge_index', 'x', 'test_mask', 'y', 'train_mask']


In [4]:
# Ver tamaños individuales
print("Matriz de features (x):", data.x.shape)
print("Aristas (edge_index):", data.edge_index.shape)
print("Etiquetas (y):", data.y.shape)


Matriz de features (x): torch.Size([203769, 165])
Aristas (edge_index): torch.Size([2, 234355])
Etiquetas (y): torch.Size([203769])


In [5]:
# Otras características
print("Nodos:", data.num_nodes)
print("Aristas:", data.num_edges)

Nodos: 203769
Aristas: 234355


El grafo provisto por la librería `torch_geometric` contiene toda la información necesaria para su construcción. Desde el código, podemos explorar fácilmente los atributos del objeto `data` (el grado en sí):

- `data.x`: matriz de características de cada nodo, con 166 features por transacción
- `data.y`: etiquetas de cada nodo (0 = legítimo, 1 = fraude, 2 = desconocido)
- `data.edge_index`: matriz con las conexiones entre nodos (aristas del grafo)
- `data.num_nodes`: cantidad total de nodos (transacciones)
- `data.num_edges`: cantidad total de aristas (conexiones entre transacciones)

Como puede verse, estos datos están organizados en estructuras separadas, y no es posible representar **toda esta información** de forma fiel en un solo archivo `.csv` plano sin perder relaciones o duplicar contenido innecesariamente.

Por eso, se opta por generar dos archivos `.csv`:
1. `nodes.csv`: contiene una fila por transacción, con su ID, etiqueta y features.
2. `edges.csv`: contiene una fila por relación, indicando cómo se conectan dos transacciones (nodos).

Esta separación permite reconstruir el grafo completo y representa correctamente la naturaleza del dataset.


Generar `nodes.csv`:

In [ ]:

# Features por transacción (nodo)
features = data.x.numpy()  # shape: (203769, 166)

# Etiquetas: 0 = legítimo, 1 = fraude, 2 = desconocido
labels = data.y.numpy()  # shape: (203769,)

# Construir DataFrame
df_nodes = pd.DataFrame(features, columns=[f'feature_{i}' for i in range(features.shape[1])])
df_nodes.insert(0, 'fraud_label', labels)
df_nodes.insert(0, 'node_id', range(data.num_nodes))

df_nodes.head()

,node_id,fraud_label,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_155,feature_156,feature_157,feature_158,feature_159,feature_160,feature_161,feature_162,feature_163,feature_164
0,0,2,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,...,-0.562153,-0.600999,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
1,1,2,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,...,0.947382,0.673103,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
2,2,2,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,...,0.670883,0.439728,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792
3,3,0,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782743,12.414557,-0.163645,...,-0.577099,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792
4,4,2,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,...,-0.511871,-0.400422,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117


In [7]:
# Exportar
df_nodes.to_csv("nodes.csv", index=False)

Generar `edges.csv`:

In [8]:
# Aristas (relaciones entre transacciones)
edge_index = data.edge_index.numpy()  # shape: (2, 234355)

df_edges = pd.DataFrame({
    'from_node': edge_index[0],
    'to_node': edge_index[1]
})

df_edges.head()

,from_node,to_node
0,0,1
1,2,3
2,4,5
3,6,7
4,8,9


In [9]:
# Exportar
df_edges.to_csv("edges.csv", index=False)

Ambos `csv` generados serán suficientes para realizar el camino inverso y lograr construir un grafo de transacciones a partir de ellos, dando a entender que tanto una tabla como un grafo son 2 maneras de representar lo mismo.

### Descripción del dataset

Continuaremos trabajando con Elliptic Data Set, presentado por Weber et al. en 2019 [1], el cual contiene datos anonimizados de 203,769 transacciones de Bitcoin. De la documentación del dataset sabemos que:

2% de las transacciones están clasificadas como ilícitas (clase 1), 21% como lícitas (clase 2) y el 77% restante no ha sido etiquetado (clase 0). 

#### Features 
Cada transacción corresponde a un intervalo temporal de 3 hs (time step). Los time step son índices del 1 al 49 y están espaciados entre sí por dos semanas. Cada time step contiene un componente de transacciones conectadas entre sí, y desconectadas del resto de los componentes. 

De cada transacción se incluyen también 166 variables (features) cuya descripción exacta no es provista, pero se dan rasgos generales:

##### Features directas de la transacción (1 a 94)
Las primeras 94 features representan información directa de la transacción. Esto incluye al time-step, la cantidad de inputs/outputs, la tarifa de transacción, el volumen de output, así como variables agregadas tales como la cantidad promedio de BTC recibido (gastado) por los inputs/outputs y el número promedio de transacciones entrantes asociadas con los inputs/outpus.

Estas son estadísticas directamente relacionadas con:
- Cómo se construyó la transacción
- De qué magnitud es
- Qué tan activa es la wallet involucrada
- Cuánto dinero se movió

##### Features agregadas de transacciones vecinas (95 a 166)
Las 72 features restantes provee estadísticas agregadas de las transacciones vecinas (una arista de distancia). Para cada variable incluida en las features directas, se calculan agregados (máximo, mínimo, desvío estándar y coeficientes de correlación) de las transacciones vecinas. 

In [10]:
# Cargar los archivos
features = pd.read_csv(r"D:\Documents\Cursos\Diplomatura FAMAF\Mentoria\elliptic-gnn-fraud-detector\data\elliptic\elliptic_txs_features.csv", header=None)
edges = pd.read_csv(r"D:\Documents\Cursos\Diplomatura FAMAF\Mentoria\elliptic-gnn-fraud-detector\data\elliptic\elliptic_txs_edgelist.csv")
classes = pd.read_csv(r"D:\Documents\Cursos\Diplomatura FAMAF\Mentoria\elliptic-gnn-fraud-detector\data\elliptic\elliptic_txs_classes.csv")

# Renombrar columnas para claridad
# features = features.rename(columns={0: "txId", 1: "time_step"})
edges = edges.rename(columns={"txId1": "source", "txId2": "target"})
classes = classes.rename(columns={"txId": "txId", "class": "label"})



In [11]:
# renombrar columnas para mayor claridad
tx_features = ["tx_feat_"+str(i) for i in range(2,95)]
agg_features = ["agg_feat_"+str(i) for i in range(1,73)]
features.columns = ["txId","time_step"] + tx_features + agg_features

In [12]:
print(f"""Shapes
{4*' '}Features : {features.shape[0]:8,} (rows)  {features.shape[1]:4,} (cols)
{4*' '}Classes  : {classes.shape[0]:8,} (rows)  {classes.shape[1]:4,} (cols)
{4*' '}Edgelist : {edges.shape[0]:8,} (rows)  {edges.shape[1]:4,} (cols)
""")



Shapes
    Features :  203,769 (rows)   167 (cols)
    Classes  :  203,769 (rows)     2 (cols)
    Edgelist :  234,355 (rows)     2 (cols)



#### Armado de dataset para modelado

In [13]:
# merge with classes
df_features = pd.merge(features, classes, left_on='txId', right_on='txId', how='left')

In [14]:
df_features['label'] = df_features['label'].apply(lambda x: '0' if x == "unknown" else x)

In [15]:
# chequeamos que las clases nos quedaron bien armadas
df_features.groupby('label').size()

label
0    157205
1      4545
2     42019
dtype: int64

## 2 - Entrenamiento de Modelos

### Preprocesamiento

In [ ]:
# Cantidad de filas con al menos un valor nulo
num_rows_with_nan = df_features.isnull().any(axis=1).sum()

# Cantidad de datos faltantes por variable
missing_per_col = df_features.isnull().sum()

(num_rows_with_nan, missing_per_col)

(np.int64(0),
 txId           0
 time_step      0
 tx_feat_2      0
 tx_feat_3      0
 tx_feat_4      0
               ..
 agg_feat_69    0
 agg_feat_70    0
 agg_feat_71    0
 agg_feat_72    0
 label          0
 Length: 168, dtype: int64)

In [17]:
# nos quedamos con datos etiquetados
data = df_features[(df_features['label']=='1') | (df_features['label']=='2')]

# creamos series para modelado
X = data[['time_step'] + tx_features+agg_features]
y = data['label'].astype(int)

In [ ]:
# división entre entrenamiento y evaluación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [19]:
# distribución de variable target en subconjuntos de train y test
print("Distribución original:", y.value_counts(normalize=True))
print("Train:", y_train.value_counts(normalize=True))
print("Test:", y_test.value_counts(normalize=True))

Distribución original: label
2    0.902392
1    0.097608
Name: proportion, dtype: float64
Train: label
2    0.902123
1    0.097877
Name: proportion, dtype: float64
Test: label
2    0.903468
1    0.096532
Name: proportion, dtype: float64


Se observa una correcta distirbución entre los subconjuntos.

### Baseline model


Se propone establecer un baseline score con un modelo simple, y luego intentamos superarlo (‘break the baseline’) con modelos más sofisticados.

Para ello se utilizará DummyClassifier de sklearn con (strategy="stratified") que predice al azar, pero manteniendo las proporciones de clases del conjunto de entrenamiento. Esto último es importante ya que se tiene un conjunto de datos fuertemente desbalanceado.

In [ ]:
# 1. Crear el modelo baseline
dummy = DummyClassifier(strategy='stratified', random_state=10)  # estrategia: predecir al azar manteniendo las proporciones

# 2. Entrenarlo con el conjunto de entrenamiento
dummy.fit(X_train, y_train)

# 3. Predecir sobre el conjunto de test
y_pred = dummy.predict(X_test)

Se defininen métricas para evaluar el modelo (y los siguientes):

In [21]:
print("\n📊 Resultados usando DummyClassifier (baseline)")
print("Accuracy:",  accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:",    recall_score(y_test, y_pred))
print("F1 score:",  f1_score(y_test, y_pred))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))


📊 Resultados usando DummyClassifier (baseline)
Accuracy: 0.8244389562976484
Precision: 0.083710407239819
Recall: 0.08231368186874305
F1 score: 0.08300616937745373
Matriz de confusión:
 [[  74  825]
 [ 810 7604]]


### Interpretación
Si bien los resultados presentan una buena accuracy, ese valor es engañoso debido al desbalance del dataset.

De ahi en mas, todas las métricas son muy bajas incluyendo un F1 pobre, lo cual es correcto, ya que ahora se tiene un baseline para romper.

## Modelos lineales


### Evaluando el poder explicativo de tx_features y agg_features

En este conjuntos de datos se cuenta con dos grupos de features: uno que corresponde a características de cada transaccion y otro que corresponde a características de su contexto inmediato. 
Para evaluar el valor explicativo de cada grupo de features, entrenamos tres tipos de modelos:
- dos modelos parciales: sólo tx_features (incluye time-step) y sólo agg_features. 
- un modelo completo (all_features)

Realizamos este análisis tanto con modelos de clasificación por regresión logística como por descenso de gradiente.  

#### Regresión logística

In [ ]:

y = data['label'].astype(int)

# División en train/test
X_tx = data[tx_features]
X_agg = data[agg_features]
X_all = data[tx_features + agg_features]

X_train_tx, X_test_tx, y_train, y_test = train_test_split(X_tx, y, test_size=0.3, random_state=10)
X_train_agg, X_test_agg = train_test_split(X_agg, test_size=0.3, random_state=10)
X_train_all, X_test_all = train_test_split(X_all, test_size=0.3, random_state=10)

# Escalado (opcional pero recomendado)
scaler_tx = StandardScaler().fit(X_train_tx)
scaler_agg = StandardScaler().fit(X_train_agg)
scaler_all = StandardScaler().fit(X_train_all)

X_train_tx = scaler_tx.transform(X_train_tx)
X_test_tx = scaler_tx.transform(X_test_tx)

X_train_agg = scaler_agg.transform(X_train_agg)
X_test_agg = scaler_agg.transform(X_test_agg)

X_train_all = scaler_all.transform(X_train_all)
X_test_all = scaler_all.transform(X_test_all)

# Entrenar modelos
def eval_model(X_train, y_train, X_test, y_test, name):
    clf = LogisticRegression(max_iter=1000)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f"\n🔹 Resultados usando {name}")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred))
    print("Recall:", recall_score(y_test, y_pred))
    print("F1 score:", f1_score(y_test, y_pred))
    print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))

# Evaluar cada grupo de features
eval_model(X_train_tx, y_train, X_test_tx, y_test, "solo features de transacción")
eval_model(X_train_agg, y_train, X_test_agg, y_test, "solo features agregadas")
eval_model(X_train_all, y_train, X_test_all, y_test, "todas las features")



🔹 Resultados usando solo features de transacción
Accuracy: 0.9076592698639943
Precision: 0.6703296703296703
Recall: 0.13232104121475055
F1 score: 0.2210144927536232
Matriz de confusión:
 [[  183  1200]
 [   90 12497]]

🔹 Resultados usando solo features agregadas
Accuracy: 0.916034359341446
Precision: 0.7536231884057971
Recall: 0.22559652928416485
F1 score: 0.34724540901502504
Matriz de confusión:
 [[  312  1071]
 [  102 12485]]

🔹 Resultados usando todas las features
Accuracy: 0.9584108804581245
Precision: 0.8369747899159664
Recall: 0.720173535791757
F1 score: 0.7741935483870968
Matriz de confusión:
 [[  996   387]
 [  194 12393]]


### Interpretación
Los resultados muestran que, al estimar una clasificación por regresión lineal, tanto las características de la transacción como de su contexto inmediato tienen bajo valor predictivo, con F1 scores de 0.22 y 0.35 respectivamente. 

Ambos modelos parciales muestran muy baja recall, indicando que no están pudiendo detectar gran parte de los casos etiquetados como fraude. 

El modelo con todas las features tiene un mejor desempeño, evidenciado en su mayor score F1, de 0.77, que se debe sobre todo a su mejor valor de recall al reducir los falsos negativos (clave en este tipo de problemas de detección de fraude). Esto indica que la combinación de las features de la transacción y de su contexto son necesarias para predecir más eficazmente los casos de fraude. 

#### Clasificador por gradiente

In [ ]:
y = data['label'].astype(int)

# Divisiones con estratificación
X_tx = data[tx_features]
X_agg = data[agg_features]
X_all = data[tx_features + agg_features]

X_train_tx, X_test_tx, y_train, y_test = train_test_split(
    X_tx, y, test_size=0.3, random_state=10, stratify=y
)
X_train_agg, X_test_agg = train_test_split(
    X_agg, test_size=0.3, random_state=10, stratify=y
)
X_train_all, X_test_all = train_test_split(
    X_all, test_size=0.3, random_state=10, stratify=y
)

# Escalado
scaler_tx = StandardScaler().fit(X_train_tx)
scaler_agg = StandardScaler().fit(X_train_agg)
scaler_all = StandardScaler().fit(X_train_all)

X_train_tx = scaler_tx.transform(X_train_tx)
X_test_tx = scaler_tx.transform(X_test_tx)

X_train_agg = scaler_agg.transform(X_train_agg)
X_test_agg = scaler_agg.transform(X_test_agg)

X_train_all = scaler_all.transform(X_train_all)
X_test_all = scaler_all.transform(X_test_all)

# Evaluación con SGDClassifier
def eval_sgd(X_train, y_train, X_test, y_test, name):
    clf = SGDClassifier(loss='log_loss', max_iter=1000, tol=1e-3, random_state=10)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f"\n🔹 Resultados usando {name} (SGDClassifier)")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred))
    print("Recall:", recall_score(y_test, y_pred))
    print("F1 score:", f1_score(y_test, y_pred))
    print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))

# Ejecutar evaluaciones
eval_sgd(X_train_tx, y_train, X_test_tx, y_test, "solo features de transacción")
eval_sgd(X_train_agg, y_train, X_test_agg, y_test, "solo features agregadas")
eval_sgd(X_train_all, y_train, X_test_all, y_test, "todas las features")



🔹 Resultados usando solo features de transacción (SGDClassifier)
Accuracy: 0.9087329992841804
Precision: 0.5973741794310722
Recall: 0.2001466275659824
F1 score: 0.29983525535420097
Matriz de confusión:
 [[  273  1091]
 [  184 12422]]

🔹 Resultados usando solo features agregadas (SGDClassifier)
Accuracy: 0.9234073013600572
Precision: 0.7370967741935484
Recall: 0.3350439882697947
F1 score: 0.46068548387096775
Matriz de confusión:
 [[  457   907]
 [  163 12443]]

🔹 Resultados usando todas las features (SGDClassifier)
Accuracy: 0.9592698639942735
Precision: 0.8545941123996432
Recall: 0.7023460410557185
F1 score: 0.7710261569416499
Matriz de confusión:
 [[  958   406]
 [  163 12443]]


### Interpretación
Nuevamente verificamos que la combinación de features de la transacción y de su contexto es lo que mejor permite predecir los casos de fraude: se logra un buen nivel de recall, logrando predecir correctamente el 70% de los casos de fraude. Al mismo tiempo, se mantiene un buen nivel de precisión, lo que indica baja proporción de falsos positivos. 

Sin embargo, vemos que en este modelo ambos modelos parciales difieren notablemente en su capacidad predictiva, siendo las features del contexto las que explican más varianza y tienen mayor poder predictivo.

#### Buscando el mejor modelo por descenso de gradiente

In [24]:
# Métricas función
def metrics(y_true, y_pred):
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
        "confusion_matrix": confusion_matrix(y_true, y_pred)
    }



In [ ]:
X_all = data[tx_features + agg_features]

# Split (sin escalar afuera)
X_train_all, X_test_all, y_train, y_test = train_test_split(
    X_all, y, test_size=0.3, random_state=10, stratify=y
)

# ========= ColumnTransformer =========
numeric_features = X_all.columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features)
])

# ========= Pipeline base =========
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SGDClassifier(random_state=10))
])

# ========= Hiperparámetros =========
param_grid = {
    'classifier__loss': ['log_loss', 'hinge', 'modified_huber'],
    'classifier__alpha': [0.0001, 0.001, 0.01],
    'classifier__learning_rate': ['constant', 'optimal', 'invscaling'],
    'classifier__eta0': [0.001, 0.01, 0.1]
}

# ========= GridSearchCV =========
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    return_train_score=True
)

grid_search.fit(X_train_all, y_train)

# Resultados: f1 promedio y varianza
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']

# Resultados
results_table = pd.DataFrame(params)
results_table['mean_f1'] = means
results_table['std_f1'] = stds

# Mejor configuración: entrenando al mejor modelo
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

# Evaluación
y_train_pred = best_estimator.predict(X_train_all)
y_test_pred = best_estimator.predict(X_test_all)


results_train = metrics(y_train, y_train_pred)
results_test = metrics(y_test, y_test_pred)

results_table.sort_values(by='mean_f1', ascending=False).head(10), best_params, results_train, results_test

(    classifier__alpha  classifier__eta0 classifier__learning_rate  \
 4              0.0001             0.001                   optimal   
 13             0.0001             0.010                   optimal   
 22             0.0001             0.100                   optimal   
 3              0.0001             0.001                   optimal   
 12             0.0001             0.010                   optimal   
 21             0.0001             0.100                   optimal   
 23             0.0001             0.100                   optimal   
 14             0.0001             0.010                   optimal   
 5              0.0001             0.001                   optimal   
 31             0.0010             0.001                   optimal   
 
    classifier__loss   mean_f1    std_f1  
 4             hinge  0.781892  0.038922  
 13            hinge  0.781892  0.038922  
 22            hinge  0.781892  0.038922  
 3          log_loss  0.758008  0.006926  
 12         l

In [26]:
# imprimir hiperparámetros del mejor modelo
print(best_params)


{'classifier__alpha': 0.0001, 'classifier__eta0': 0.001, 'classifier__learning_rate': 'optimal', 'classifier__loss': 'hinge'}


In [27]:
# Evaluar el modelo entrenado
print("\n📊 Evaluación en conjunto de TEST (modelo final)")
print("Accuracy:", accuracy_score(y_test, y_test_pred))
print("Precision:", precision_score(y_test, y_test_pred))
print("Recall:", recall_score(y_test, y_test_pred))
print("F1 Score:", f1_score(y_test, y_test_pred))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_test_pred))



📊 Evaluación en conjunto de TEST (modelo final)
Accuracy: 0.9616320687186829
Precision: 0.8404605263157895
Recall: 0.749266862170088
F1 Score: 0.7922480620155039
Matriz de confusión:
 [[ 1022   342]
 [  194 12412]]


### Interpretación 
El mejor modelo encontrado es un clasificador tipo SVM lineal (SGDClassifier(loss='hinge')) entrenado con SGD, con baja regularización (alpha = 0.0001) y un learning rate adaptativo.

Se obtiene un F1 score de 0.79, indicando un buen equilibrio entre recall y precisión. 

El modelo tiene un recall de 0.75, indicando que se detectan tres cuartas partes de los casos de fraude. 

## Decision tree

In [ ]:
# Etiquetas y features combinadas
y = data['label'].astype(int)
X = data[tx_features + agg_features]

# División (sin escalado)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10, stratify=y)

# Configuraciones a probar
configs = [
    {"n_estimators": 50, "max_depth": 10, "min_samples_split": 2},
    {"n_estimators": 100, "max_depth": 20, "min_samples_split": 5},
    {"n_estimators": 150, "max_depth": None, "min_samples_split": 2},
    {"n_estimators": 100, "max_depth": 15, "min_samples_split": 10},
    {"n_estimators": 200, "max_depth": 30, "min_samples_split": 3},
]

# Evaluar Random Forest para cada configuración
resultados = []

print("\n🌟 Evaluación con TODAS LAS FEATURES (Random Forest)\n")

for i, config in enumerate(configs):
    clf = RandomForestClassifier(
        n_estimators=config['n_estimators'],
        max_depth=config['max_depth'],
        min_samples_split=config['min_samples_split'],
        random_state=10,
        n_jobs=-1
    )
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print(f"🌲 Modelo {i+1} - Config: {config}")
    print(f"Accuracy: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | F1: {f1:.4f}")
    print("Matriz de confusión:\n", cm, "\n")

    resultados.append({
        "modelo": f"RF_{i+1}",
        "config": config,
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1
    })

# Mostrar resumen final
df_resultados = pd.DataFrame(resultados)
display(df_resultados)



🌟 Evaluación con TODAS LAS FEATURES (Random Forest)

🌲 Modelo 1 - Config: {'n_estimators': 50, 'max_depth': 10, 'min_samples_split': 2}
Accuracy: 0.9843 | Precision: 0.9957 | Recall: 0.8424 | F1: 0.9126
Matriz de confusión:
 [[ 1149   215]
 [    5 12601]] 

🌲 Modelo 2 - Config: {'n_estimators': 100, 'max_depth': 20, 'min_samples_split': 5}
Accuracy: 0.9873 | Precision: 0.9966 | Recall: 0.8724 | F1: 0.9304
Matriz de confusión:
 [[ 1190   174]
 [    4 12602]] 

🌲 Modelo 3 - Config: {'n_estimators': 150, 'max_depth': None, 'min_samples_split': 2}
Accuracy: 0.9875 | Precision: 0.9958 | Recall: 0.8754 | F1: 0.9317
Matriz de confusión:
 [[ 1194   170]
 [    5 12601]] 

🌲 Modelo 4 - Config: {'n_estimators': 100, 'max_depth': 15, 'min_samples_split': 10}
Accuracy: 0.9864 | Precision: 0.9966 | Recall: 0.8636 | F1: 0.9254
Matriz de confusión:
 [[ 1178   186]
 [    4 12602]] 

🌲 Modelo 5 - Config: {'n_estimators': 200, 'max_depth': 30, 'min_samples_split': 3}
Accuracy: 0.9873 | Precision: 0.9958

,modelo,config,accuracy,precision,recall,f1
0,RF_1,"{'n_estimators': 50, 'max_depth': 10, 'min_sam...",0.984252,0.995667,0.842375,0.912629
1,RF_2,"{'n_estimators': 100, 'max_depth': 20, 'min_sa...",0.987258,0.996650,0.872434,0.930414
2,RF_3,"{'n_estimators': 150, 'max_depth': None, 'min_...",0.987473,0.995830,0.875367,0.931721
3,RF_4,"{'n_estimators': 100, 'max_depth': 15, 'min_sa...",0.986399,0.996616,0.863636,0.925373
4,RF_5,"{'n_estimators': 200, 'max_depth': 30, 'min_sa...",0.987258,0.995819,0.873167,0.930469


### Interpretación 

Se entrenaron 5 modelos `RandomForestClassifier` con diferentes combinaciones de hiperparámetros, utilizando **todas las features disponibles** (transaccionales + agregadas), sin aplicar normalización ya que no es necesaria en este caso.

Las combinaciones evaluadas incluyeron variaciones en:
- `n_estimators` (cantidad de árboles en el bosque)
- `max_depth` (profundidad máxima por árbol)
- `min_samples_split` (mínimo de muestras para dividir un nodo)

Todos tuvieron muy buenos resultados, destacándose:
| RF_3 (150 árboles, sin límite de profundidad) |

Este modelo logró el **mayor F1 score**, lo cual indica un equilibrio óptimo entre:
- **Precision muy alta** (casi no hay falsos positivos)
- **Recall elevado** (detecta la mayoría de los fraudes reales)


En comparación con modelos lineales como `SGDClassifier` (SVM lineal), Random Forest mostró un rendimiento superior, especialmente en **recall** y **F1 score**. Esto se debe a que:

- Random Forest **no asume relaciones lineales** entre variables, por lo que puede capturar patrones más complejos en los datos.
- Al combinar múltiples árboles, reduce el sobreajuste y generaliza mejor.
- Es menos sensible a la escala de los datos y al ruido, lo que simplifica el preprocesamiento.
- Se adapta naturalmente a problemas con múltiples interacciones entre variables, como ocurre en la detección de fraude.

## Conclusión

Random Forest demostró ser una **línea base sólida** para abordar la clasificación de fraudes con este dataset. El modelo `RF_3` será utilizado como referencia para futuras entregas, en las que se explorarán enfoques más avanzados como Graph Neural Networks (GNNs).


## Referencias


1. M. Weber, G. Domeniconi, J. Chen, D. K. I. Weidele, C. Bellei, T. Robinson, C. E. Leiserson, "Anti-Money Laundering in Bitcoin: Experimenting with Graph Convolutional Networks for Financial Forensics", KDD ’19 Workshop on Anomaly Detection in Finance, August 2019, Anchorage, AK, USA.